In [3]:
import girder_client 
import json
import jsonschema
from jsonschema import validate

%load_ext autoreload
%autoreload 2
import adrcSchemaHelpers as hlprs
    
gc = girder_client.GirderClient(apiUrl="https://styx.neurology.emory.edu/girder/api/v1")
_ = gc.authenticate(interactive=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Login or email: admin
Password for admin: ········


In [4]:
# sampPatient = '638e33521f75016b81fdb8d2' 
# itemSet = list(gc.getResource("resource/%s/items?type=folder&limit=1000" % sampPatient))

In [5]:
# dsaNpSchema = {
#     "type": "object",
#     "properties": {
#         "blockID": {"type": "string"},
#         "caseID": {"type": "string", "pattern": "^(E|A)\d\d-\d+"},
#         "stainID": {"type": "string", "enum": ["HE","Tau","aBeta","Syn","p62","pTDP","Biels","Ubiq","LFB"]},
#     },
# }

import json

with open("./adrcNpSchema.json","r") as fp:
    dsaNpSchema = json.load(fp)

### Additional properties https://support.riverbed.com/apis/steelscript/reschema/jsonschema.html


def validateNPJson(jsonData):
    try:
        validate(instance=jsonData, schema=dsaNpSchema)
    except jsonschema.exceptions.ValidationError as err:
        return False
    return True

# for i in itemSet:
#     if 'meta' in i:
#         if 'npSchema' in i['meta']:
#             npSchema = i['meta']['npSchema']
#             isvalid = validateNPJson(npSchema)
#             if not isvalid:
#                 print(npSchema)

In [8]:
yearsToScan = ['2020','2019','2018','2017'] ### Hard coding some years so I can focus on a smaller subset

validated = 0
for f in gc.listFolder('6104244568f4fe34fca16b19',parentFolderType='collection'):
    if f['name'] in yearsToScan:
        print("YEAR=============",f['name'],"=================")
        ## GEt the subfolders..
        for sf in gc.listFolder(f['_id']):
            #http://candygram.neurology.emory.edu:8080/api/v1#!/resource/resource_getResourceItems_id_items
            itemSet = list(gc.getResource("resource/%s/items?type=folder&limit=1000" % sf['_id']))
    #         hlprs.scanMetadata(gc,sf,updateGirder=True)
            ## This is where we can modify the scanMetadata file .. if we bundle in the validateNPJson function
            ## then we can ONLY update npSchema with valid data, and if the data is NOT valid we probably keep the data
            ## but stick it on like  rawSlideMetadata or whatever key you decide you like
    



# for f in gc.listFolder('638e2da11f75016b81fda12f',parentFolderType='collection'):
    
#     if f['name'] in yearsToScan:
    

#         print("YEAR=============",f['name'],"=================")
#         ## GEt the subfolders..
#         for sf in gc.listFolder(f['_id']):
#             #http://candygram.neurology.emory.edu:8080/api/v1#!/resource/resource_getResourceItems_id_items
#             itemSet = list(gc.getResource("resource/%s/items?type=folder&limit=1000" % sf['_id']))
#     #         hlprs.scanMetadata(gc,sf,updateGirder=True)
#             ## This is where we can modify the scanMetadata file .. if we bundle in the validateNPJson function
#             ## then we can ONLY update npSchema with valid data, and if the data is NOT valid we probably keep the data
#             ## but stick it on like  rawSlideMetadata or whatever key you decide you like
#             for i in itemSet:
#                 if 'meta' in i:
#                     if 'npSchema' in i['meta']:
#                         npSchema = i['meta']['npSchema']
#                         isvalid = validateNPJson(npSchema)
#                         if not isvalid:
#                             print(npSchema)    


YEAR============= 2017 =================
{'artifacts': 'no', 'bad_scan': 'no', 'control': 'unknown', 'ignore': 'yes', 'jc_confirm': 'yes', 'notes': 'none', 'region': 'unknown', 'stain': 'unknown', 'subject': 'unknown'}
{'artifacts': 'no', 'bad_scan': 'no', 'control': 'unknown', 'ignore': 'yes', 'jc_confirm': 'yes', 'notes': 'none', 'region': 'unknown', 'stain': 'unknown', 'subject': 'unknown'}
{'artifacts': 'no', 'bad_scan': 'no', 'control': 'unknown', 'ignore': 'no', 'jc_confirm': 'no', 'notes': 'none', 'region': '', 'stain': '', 'subject': ''}
{'artifacts': 'no', 'bad_scan': 'no', 'control': 'unknown', 'ignore': 'no', 'jc_confirm': 'no', 'notes': 'none', 'npSchema': {'blockID': '19', 'caseID': 'E17-128', 'regionName': 'Spinal cord', 'stainID': 'Stain ID Not Valid'}, 'region': '', 'stain': '', 'subject': ''}
{'artifacts': 'no', 'bad_scan': 'no', 'control': 'unknown', 'ignore': 'no', 'jc_confirm': 'no', 'notes': 'none', 'npSchema': {'blockID': '20', 'caseID': 'E17-128', 'regionName': '

{'artifacts': 'no', 'bad_scan': 'no', 'control': 'unknown', 'ignore': 'no', 'jc_confirm': 'no', 'notes': 'none', 'npSchema': {'blockID': '17', 'caseID': 'E18-61', 'regionName': 'Upper spinal cord', 'stainID': 'Stain ID Not Valid'}, 'region': '', 'stain': '', 'subject': ''}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'artifacts': 'no', 'bad_scan': 'no', 'brain_region': 'unknown', 'control': 'no', 'failed_scan': 'no', 'ignore': 'no', 'jc_confirm': 'no', 'notes': 'none', 'npSchema': {'blockID': '1', 'caseID': 'E18-92', 'regionName': 'Hippocampus', 'stainID': 'Stain ID Not Valid'}, 'region': 'A1', 'stain': 'CP13', 'subject': 'A18-92'}
{'artifacts': 'no', 'bad_scan': 'no', 'brain_region': 'unknown', 'control': 'no', 'failed_scan': 'no', 'ignore': 'no', 'jc_confirm': 'no', 'notes': 'none', 'npSchema': {'blockID': '9', 'caseID': 'E18-92', 'regionName': 'Insular cortex', 'stainID': 'Stain ID Not Valid'}, 'region': 'A9', 'stain': 'CP13', 'subject': 'A18-92'}
{'artifacts': 'no',

{}
{'npSchema': {'blockID': '19', 'caseID': 'E19-34', 'regionName': 'Mid-level spinal cord', 'stainID': 'Stain ID Not Valid'}}
{'npSchema': {'blockID': '20', 'caseID': 'E19-34', 'regionName': 'Lower spinal cord', 'stainID': 'Stain ID Not Valid'}}
{}
{'npSchema': {'blockID': '18', 'caseID': 'E19-70', 'regionName': 'Upper spinal cord', 'stainID': 'Stain ID Not Valid'}}
{'npSchema': {'blockID': '19', 'caseID': 'E19-70', 'regionName': 'Mid-level spinal cord', 'stainID': 'Stain ID Not Valid'}}
{'npSchema': {'blockID': '1', 'caseID': 'E19-70', 'regionName': 'Left Hippocampus', 'stainID': 'Stain ID Not Valid'}}
{'npSchema': {'blockID': '20', 'caseID': 'E19-70', 'regionName': 'Lower spinal cord', 'stainID': 'Stain ID Not Valid'}}
{'npSchema': {'blockID': '21', 'caseID': 'E19-70', 'regionName': 'Dorsal roots', 'stainID': 'Stain ID Not Valid'}}
{'npSchema': {'blockID': '22', 'caseID': 'E19-70', 'regionName': 'Ventral roots', 'stainID': 'Stain ID Not Valid'}}
{'npSchema': {'blockID': '23', 'caseI